In [1]:
import os
path = "/Users/patricia/Documents/code/python-code/behavior-detection/src"
os.chdir(path)

from core.preprocessors.data_loader import DataLoader

# Load data
file_path = '../data/new_logs_labels.csv'
df = DataLoader.load_data(file_path, delimiter=';')

In [2]:
df.head()

,id_log,aluno,grupo,num_dia,num_log,log_type,ultimo_passo_correto,verificado_com_mouse,verificado_com_teclado,idle_time_acumulado,...,comportamento_off_task,comportamento_on_system,comportamento_indefinido,ultimo_comportamento,ultimo_comportamento_on_task,ultimo_comportamento_on_task_conversation,ultimo_comportamento_on_task_out,ultimo_comportamento_off_task,ultimo_comportamento_on_system,ultimo_comportamento_indefinido
0,8224,1,2,1,1,step_verification,0,0,1,0,...,0,0,0,?,0,0,0,0,0,1
1,527786,1,2,1,2,user_idle,1,0,0,2,...,0,0,0,ON TASK,1,0,0,0,0,0
2,527787,1,2,1,3,user_idle,0,0,0,4,...,0,0,0,ON TASK,1,0,0,0,0,0
3,527788,1,2,1,4,user_idle,0,0,0,6,...,0,0,0,ON TASK,1,0,0,0,0,0
4,527789,1,2,1,5,user_idle,0,0,0,8,...,0,0,0,ON TASK,1,0,0,0,0,0


In [21]:
!pip install ydata-profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.3/657.3 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 9.9 MB/s eta 0:00:00:00:010:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=a44170b60d995e9ecf195e0e77860142ea31485547b5ffe081051ce1895ff153
  Stored in directory: /Users/patricia/Library/Caches/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0


In [ ]:
def analyze_dataframe(df):
    from ydata_profiling import ProfileReport

    profile = ProfileReport(df, title="Dataset Analysis Report",
                            minimal=False,
                            correlations={
                                "pearson": {"calculate": True},
                                "spearman": {"calculate": True},
                                "kendall": {"calculate": True},
                                "phi_k": {"calculate": True},
                            })
    return profile


# Uso
profile = analyze_dataframe(df)
profile.to_file("dataset_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
from core.exploration.data_exploration import DataExploration

# Criar arquivo de metadados
DataExploration.create_metadata_file(df, '../data/metadata.csv')

Metadados salvos em ../data/metadata.csv


In [10]:
# Verificar valores únicos e a quantidade de cada um nas colunas especificadas
colunas = ['grupo', 'aluno', 'num_dia']
for coluna in colunas:
    print(f"Valores únicos e contagem para a coluna '{coluna}':")
    print(df[coluna].nunique())
    print(df[coluna].unique())
    print("\n")

print(df.groupby(['grupo', 'aluno']).ngroups)
print(df.groupby(['grupo', 'aluno']).size())

Valores únicos e contagem para a coluna 'grupo':
10
[ 2  5  3  1  7  6  9  8 10  4]


Valores únicos e contagem para a coluna 'aluno':
30
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]


Valores únicos e contagem para a coluna 'num_dia':
10
[ 1  2  3  4  5  6  7  8  9 10]


30
grupo  aluno
1      4        173
       14       185
       27       195
2      1        184
       20       192
       25       149
3      3        191
       11       283
       22       177
4      10       189
       12       186
       29       161
5      2        188
       18       188
       24       159
6      6        167
       19       158
       21       185
7      5        181
       15       145
       30       199
8      8        164
       17       183
       23       214
9      7        202
       13       186
       26       166
10     9        188
       16       205
       28       182
dtype: int64


In [13]:
# Verificar se há valores repetidos na coluna 'aluno' em diferentes valores da coluna 'grupo'
duplicated_alunos = df[df.duplicated(subset=['aluno', 'grupo'], keep=False)].sort_values(['aluno', 'grupo'])
print("\nStudents that appear in different groups:")
print(df.groupby('aluno')['grupo'].unique()[df.groupby('aluno')['grupo'].transform('size') > 1])


Students that appear in different groups:
aluno
1      [2]
2      [5]
3      [3]
4      [1]
5      [7]
6      [6]
7      [9]
8      [8]
9     [10]
10     [4]
11     [3]
12     [4]
13     [9]
14     [1]
15     [7]
16    [10]
17     [8]
18     [5]
19     [6]
20     [2]
21     [6]
22     [3]
23     [8]
24     [5]
25     [2]
26     [9]
27     [1]
28    [10]
29     [4]
30     [7]
Name: grupo, dtype: object


In [20]:
# Verificar se há valores repetidos nas colunas especificadas
print("\nLogs that appear with duplicate combinations:")
print(df.groupby(['aluno','num_log']).size()[df.groupby(['aluno','num_log']).size() > 1])


Logs that appear with duplicate combinations:
Series([], dtype: int64)


In [4]:
from core.preprocessors.data_splitter import DataSplitter

# Conjunto de treinamento
X, y = DataSplitter.split_into_x_y(df, 'comportamento')

In [5]:
# Exibir informações sobre X e y
print("Colunas de X:")
print(X.columns)
print("\nColunas de y:")
print(y)


Colunas de X:
Index(['id_log', 'aluno', 'grupo', 'num_dia', 'num_log', 'log_type',
       'ultimo_passo_correto', 'verificado_com_mouse',
       'verificado_com_teclado', 'idle_time_acumulado',
       ...
       'comportamento_off_task', 'comportamento_on_system',
       'comportamento_indefinido', 'ultimo_comportamento',
       'ultimo_comportamento_on_task',
       'ultimo_comportamento_on_task_conversation',
       'ultimo_comportamento_on_task_out', 'ultimo_comportamento_off_task',
       'ultimo_comportamento_on_system', 'ultimo_comportamento_indefinido'],
      dtype='object', length=371)

Colunas de y:
0                    ON TASK
1                    ON TASK
2                    ON TASK
3                    ON TASK
4                    ON TASK
                ...         
5520    ON TASK CONVERSATION
5521    ON TASK CONVERSATION
5522    ON TASK CONVERSATION
5523    ON TASK CONVERSATION
5524    ON TASK CONVERSATION
Name: comportamento, Length: 5525, dtype: object


In [6]:
# Inspecionar y
DataExploration.inspect_dataframe(y)
DataExploration.inspect_categorical_data(y)

Primeiras linhas do DataFrame:
0    ON TASK
1    ON TASK
2    ON TASK
3    ON TASK
4    ON TASK
Name: comportamento, dtype: object

Informações sobre o DataFrame:
<class 'pandas.core.series.Series'>
RangeIndex: 5525 entries, 0 to 5524
Series name: comportamento
Non-Null Count  Dtype 
--------------  ----- 
5525 non-null   object
dtypes: object(1)
memory usage: 43.3+ KB
None
Número de instâncias por classe:
    Categoria: ON TASK, Contagem: 3159
    Categoria: ON SYSTEM, Contagem: 907
    Categoria: ON TASK OUT, Contagem: 380
    Categoria: OFF TASK, Contagem: 629
    Categoria: ?, Contagem: 36
    Categoria: ON TASK CONVERSATION, Contagem: 414
